In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda
import math
import time

print("⚠️  CUDA C++ is PRIMARY. Python/Numba for quick testing only.")
if cuda.is_available():
    print(f"GPU: {cuda.get_current_device().name}")

---

### 🔷 CUDA C++ Implementation (Primary)

## Part 1: Case Study - Softmax Optimization

### The Problem

Softmax is ubiquitous in deep learning:

```
softmax(x_i) = exp(x_i - max(x)) / sum(exp(x_j - max(x)))

Three passes needed:
1. Find max(x)           - Reduction
2. Compute sum(exp(...)) - Map + Reduction  
3. Divide by sum         - Map
```

### CUDA C++ Optimization Journey

The following code demonstrates the optimization journey from naive to online algorithm:
- **Version 0**: Naive (three separate kernels)
- **Version 1**: Fused (single kernel, multiple passes)
- **Version 2**: Online algorithm (single pass using running max/sum)

In [ ]:
%%writefile softmax_optimization.cu
// softmax_optimization.cu - Complete optimization case study
#include <stdio.h>
#include <cuda_runtime.h>
#include <float.h>
#include <math.h>

// Version 0: Naive (three separate kernels)
__global__ void findMax(const float* input, float* maxVal, int n) {
    __shared__ float sdata[256];
    int tid = threadIdx.x;
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Load and find max
    sdata[tid] = (i < n) ? input[i] : -FLT_MAX;
    __syncthreads();
    
    // Reduce
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] = fmaxf(sdata[tid], sdata[tid + s]);
        }
        __syncthreads();
    }
    
    if (tid == 0) atomicMax((int*)maxVal, __float_as_int(sdata[0]));
}

__global__ void computeExpSum(const float* input, float maxVal, 
                              float* expSum, int n) {
    __shared__ float sdata[256];
    int tid = threadIdx.x;
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    
    sdata[tid] = (i < n) ? expf(input[i] - maxVal) : 0.0f;
    __syncthreads();
    
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) sdata[tid] += sdata[tid + s];
        __syncthreads();
    }
    
    if (tid == 0) atomicAdd(expSum, sdata[0]);
}

__global__ void applySoftmax(const float* input, float maxVal, 
                             float expSum, float* output, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        output[i] = expf(input[i] - maxVal) / expSum;
    }
}

// Version 1: Fused (single pass per row)
__global__ void softmaxFused(const float* input, float* output, 
                             int rows, int cols) {
    extern __shared__ float sdata[];
    
    int row = blockIdx.x;
    int tid = threadIdx.x;
    const float* rowIn = input + row * cols;
    float* rowOut = output + row * cols;
    
    // Step 1: Find max (single pass over row)
    float localMax = -FLT_MAX;
    for (int i = tid; i < cols; i += blockDim.x) {
        localMax = fmaxf(localMax, rowIn[i]);
    }
    sdata[tid] = localMax;
    __syncthreads();
    
    // Reduce to find row max
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) sdata[tid] = fmaxf(sdata[tid], sdata[tid + s]);
        __syncthreads();
    }
    float rowMax = sdata[0];
    __syncthreads();
    
    // Step 2: Compute exp and sum
    float localSum = 0.0f;
    for (int i = tid; i < cols; i += blockDim.x) {
        localSum += expf(rowIn[i] - rowMax);
    }
    sdata[tid] = localSum;
    __syncthreads();
    
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) sdata[tid] += sdata[tid + s];
        __syncthreads();
    }
    float rowSum = sdata[0];
    __syncthreads();
    
    // Step 3: Apply softmax
    for (int i = tid; i < cols; i += blockDim.x) {
        rowOut[i] = expf(rowIn[i] - rowMax) / rowSum;
    }
}

// Version 2: Online softmax (single pass)
__global__ void softmaxOnline(const float* input, float* output,
                              int rows, int cols) {
    extern __shared__ float sdata[];
    float* smaxes = sdata;
    float* ssums = sdata + blockDim.x;
    
    int row = blockIdx.x;
    int tid = threadIdx.x;
    const float* rowIn = input + row * cols;
    float* rowOut = output + row * cols;
    
    // Online algorithm: track max and sum in single pass
    float localMax = -FLT_MAX;
    float localSum = 0.0f;
    
    for (int i = tid; i < cols; i += blockDim.x) {
        float val = rowIn[i];
        if (val > localMax) {
            // Rescale existing sum
            localSum *= expf(localMax - val);
            localMax = val;
        }
        localSum += expf(val - localMax);
    }
    
    smaxes[tid] = localMax;
    ssums[tid] = localSum;
    __syncthreads();
    
    // Reduce (combining max and sum together)
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            float maxA = smaxes[tid], maxB = smaxes[tid + s];
            float sumA = ssums[tid], sumB = ssums[tid + s];
            
            if (maxA > maxB) {
                ssums[tid] = sumA + sumB * expf(maxB - maxA);
                smaxes[tid] = maxA;
            } else {
                ssums[tid] = sumA * expf(maxA - maxB) + sumB;
                smaxes[tid] = maxB;
            }
        }
        __syncthreads();
    }
    
    float rowMax = smaxes[0];
    float rowSum = ssums[0];
    __syncthreads();
    
    // Apply softmax
    for (int i = tid; i < cols; i += blockDim.x) {
        rowOut[i] = expf(rowIn[i] - rowMax) / rowSum;
    }
}

int main() {
    int rows = 1024, cols = 4096;
    size_t size = rows * cols * sizeof(float);
    
    float *h_input = (float*)malloc(size);
    float *h_output = (float*)malloc(size);
    
    // Initialize
    for (int i = 0; i < rows * cols; i++) {
        h_input[i] = (float)(rand() % 100) / 10.0f - 5.0f;
    }
    
    float *d_input, *d_output;
    cudaMalloc(&d_input, size);
    cudaMalloc(&d_output, size);
    cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);
    
    // Benchmark each version
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    int blockSize = 256;
    int sharedSize = blockSize * sizeof(float);
    
    // Version 1: Fused
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        softmaxFused<<<rows, blockSize, sharedSize>>>(d_input, d_output, rows, cols);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float fusedTime;
    cudaEventElapsedTime(&fusedTime, start, stop);
    
    // Version 2: Online
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        softmaxOnline<<<rows, blockSize, 2 * sharedSize>>>(d_input, d_output, rows, cols);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float onlineTime;
    cudaEventElapsedTime(&onlineTime, start, stop);
    
    printf("=== Softmax Optimization Results ===\n");
    printf("Matrix: %d x %d\n", rows, cols);
    printf("Fused (3 passes):  %.2f ms\n", fusedTime / 100);
    printf("Online (1 pass):   %.2f ms\n", onlineTime / 100);
    printf("Speedup: %.2fx\n", fusedTime / onlineTime);
    
    // Calculate bandwidth
    float bytes = 2.0f * rows * cols * sizeof(float); // Read + write
    printf("Online bandwidth: %.1f GB/s\n", 
           bytes / (onlineTime / 100 * 1e6));
    
    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    free(h_output);
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -O3 -o softmax_optimization softmax_optimization.cu
!./softmax_optimization

---

## Part 2: Optimization Stages

### Stage-by-Stage Analysis

```
┌──────────────────────────────────────────────────────────────────┐
│ Version 0: Naive (3 separate kernels)                           │
├──────────────────────────────────────────────────────────────────┤
│ Problem: 3 kernel launches, 3 global memory passes              │
│ Memory: Read input 3x, write output 1x + intermediates          │
│ Launch: Overhead for 3 kernels × rows                           │
└──────────────────────────────────────────────────────────────────┘
                              ↓
┌──────────────────────────────────────────────────────────────────┐
│ Version 1: Fused (1 kernel, 3 passes per row)                   │
├──────────────────────────────────────────────────────────────────┤
│ Improvement: Single kernel, row-level parallelism               │
│ Memory: Still 3 passes over row data                            │
│ Problem: Re-reading row 3 times from global memory              │
└──────────────────────────────────────────────────────────────────┘
                              ↓
┌──────────────────────────────────────────────────────────────────┐
│ Version 2: Online Algorithm (1 pass)                            │
├──────────────────────────────────────────────────────────────────┤
│ Improvement: Compute max and sum simultaneously!                │
│ Memory: Read input 2x (once for max+sum, once for output)       │
│ Trick: Rescale running sum when new max found                   │
└──────────────────────────────────────────────────────────────────┘
```

### 🔶 Python/Numba (Optional - Quick Testing)

In [ ]:
# Python/Numba Implementation (OPTIONAL - for verification)

@cuda.jit
def softmax_naive(input_arr, output_arr, n):
    """Naive: Each thread handles one element (bad!)"""
    i = cuda.grid(1)
    if i >= n:
        return
    
    # Each thread finds global max (terrible!)
    max_val = input_arr[0]
    for j in range(n):
        if input_arr[j] > max_val:
            max_val = input_arr[j]
    
    # Each thread computes sum (terrible!)
    exp_sum = 0.0
    for j in range(n):
        exp_sum += math.exp(input_arr[j] - max_val)
    
    # Each thread writes its element
    output_arr[i] = math.exp(input_arr[i] - max_val) / exp_sum

@cuda.jit
def softmax_shared(input_arr, output_arr, n):
    """Better: Use shared memory for reductions"""
    sdata = cuda.shared.array(256, dtype=np.float32)
    
    tid = cuda.threadIdx.x
    i = cuda.blockIdx.x * cuda.blockDim.x + tid
    
    # Load to shared memory
    sdata[tid] = input_arr[i] if i < n else -1e38
    cuda.syncthreads()
    
    # Find block max
    s = cuda.blockDim.x // 2
    while s > 0:
        if tid < s and sdata[tid + s] > sdata[tid]:
            sdata[tid] = sdata[tid + s]
        cuda.syncthreads()
        s //= 2
    
    block_max = sdata[0]
    cuda.syncthreads()
    
    # Compute exp sum
    sdata[tid] = math.exp(input_arr[i] - block_max) if i < n else 0.0
    cuda.syncthreads()
    
    s = cuda.blockDim.x // 2
    while s > 0:
        if tid < s:
            sdata[tid] += sdata[tid + s]
        cuda.syncthreads()
        s //= 2
    
    exp_sum = sdata[0]
    
    if i < n:
        output_arr[i] = math.exp(input_arr[i] - block_max) / exp_sum

# Test
n = 256
x = np.random.randn(n).astype(np.float32)
y_gpu = np.zeros_like(x)

d_x = cuda.to_device(x)
d_y = cuda.to_device(y_gpu)

softmax_shared[1, 256](d_x, d_y, n)
y_gpu = d_y.copy_to_host()

# Compare with NumPy
y_cpu = np.exp(x - np.max(x)) / np.sum(np.exp(x - np.max(x)))

print(f"Max error: {np.max(np.abs(y_gpu - y_cpu)):.2e}")
print(f"Sum of softmax: {np.sum(y_gpu):.6f} (should be 1.0)")

---

## Part 3: Profiling-Driven Optimization

### Optimization Workflow

```
┌─────────────────────────────────────────────────────────────────┐
│                    PROFILING WORKFLOW                          │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1. BASELINE                                                    │
│     └─ Run ncu on naive version                                 │
│     └─ Record: time, bandwidth, occupancy, bottleneck           │
│                                                                 │
│  2. IDENTIFY BOTTLENECK                                         │
│     ├─ Memory bound? → Improve access patterns                  │
│     ├─ Compute bound? → Reduce instructions                     │
│     └─ Latency bound? → Increase occupancy                      │
│                                                                 │
│  3. OPTIMIZE                                                    │
│     └─ Apply ONE optimization                                   │
│     └─ Verify correctness!                                      │
│                                                                 │
│  4. MEASURE                                                     │
│     └─ Re-profile                                               │
│     └─ Compare metrics                                          │
│     └─ If improved, keep. If not, revert.                       │
│                                                                 │
│  5. REPEAT until satisfied                                      │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Key Metrics to Track

```cpp
// Profile command for each version
// ncu --set full -o softmax_v0 ./softmax_opt  # Baseline
// ncu --set full -o softmax_v1 ./softmax_opt  # After opt 1

/*
Track these metrics across versions:

| Metric                  | V0     | V1     | V2     |
|-------------------------|--------|--------|--------|
| Time (ms)               | 5.2    | 2.1    | 1.4    |
| Memory Throughput (%)   | 45%    | 72%    | 89%    |
| L2 Hit Rate             | 12%    | 35%    | 68%    |
| Occupancy               | 25%    | 50%    | 75%    |
| Achieved Bandwidth      | 180    | 290    | 380    |
| Theoretical BW          | 400    | 400    | 400    |
| Efficiency              | 45%    | 73%    | 95%    |
*/
```

---

## Part 4: Complete Optimization Checklist

### The GPU Optimization Checklist

```
□ MEMORY ACCESS
  ├─ □ Coalesced global memory access?
  ├─ □ Shared memory for reuse?
  ├─ □ Bank conflict free?
  └─ □ Minimize global memory traffic?

□ PARALLELISM
  ├─ □ Sufficient threads (>= 10K)?
  ├─ □ Good occupancy (>= 50%)?
  ├─ □ Load balanced work?
  └─ □ Avoid thread divergence?

□ KERNEL DESIGN
  ├─ □ Fused operations (reduce launches)?
  ├─ □ Grid-stride loops for flexibility?
  ├─ □ Warp primitives where applicable?
  └─ □ Appropriate block size?

□ SYNCHRONIZATION
  ├─ □ Minimize __syncthreads()?
  ├─ □ Use warp-level sync when possible?
  └─ □ Atomics only when necessary?

□ PROFILING
  ├─ □ Identified bottleneck?
  ├─ □ Measured before/after?
  └─ □ Compared to theoretical peak?
```

---

## Exercises

### Exercise 1: LayerNorm Optimization

Apply the same optimization journey to LayerNorm:

```cpp
// LayerNorm: y = (x - mean) / sqrt(var + eps) * gamma + beta

// TODO: Implement these versions:
// V0: Separate kernels for mean, variance, normalize
// V1: Fused single kernel
// V2: Online algorithm (Welford's method)

__global__ void layernormOnline(const float* input, float* output,
                                const float* gamma, const float* beta,
                                int rows, int cols, float eps) {
    // Welford's online algorithm for mean and variance
    // Your implementation here
}
```

### Exercise 2: Profile and Document

For your softmax implementation:
1. Run `ncu` on each version
2. Create a table of metrics
3. Identify the bottleneck at each stage
4. Calculate achieved vs theoretical bandwidth

### Exercise 3: Vectorized Memory Access

```cpp
// Add vectorized loads to improve bandwidth
__global__ void softmaxVectorized(const float4* input, float4* output,
                                  int rows, int cols) {
    // Load 4 floats at once with float4
    // Process and store
}
```

---

## Key Takeaways

```
┌─────────────────────────────────────────────────────────────────┐
│                  OPTIMIZATION WORKFLOW                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  1. Profile first - never guess!                                │
│  2. One optimization at a time                                  │
│  3. Measure before and after                                    │
│  4. Document your journey                                       │
│  5. Compare to theoretical limits                               │
│                                                                 │
│  Key Techniques Used:                                           │
│  • Kernel fusion (reduce launches)                              │
│  • Online algorithms (reduce passes)                            │
│  • Shared memory (reduce global access)                         │
│  • Warp primitives (reduce sync overhead)                       │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

## Next Steps

**Day 5:** Practice & checkpoint quiz for Week 8!

**Week 9:** We'll explore CUDA Streams & Concurrency for overlapping computation with data transfer.